In [1]:
#Importing libraries
%pip install -U textblob
%pip install vaderSentiment
# python -m textblob.download_corpora
%pip install tabulate
%pip install spacy
%pip install gensim
%pip install --upgrade huggingface_hub
%pip install transformers
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import pickle
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from PIL import Image
import streamlit as st
import pickle as pk
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import string  # Add this import statement
from nltk.stem.porter import PorterStemmer

# Creating Recipe Dataframe

In [4]:
recipe_data = pd.read_csv('D:/6. Books_MSBA/8. NLP/22. Project/recipe_data_filtered.csv')

In [5]:
recipe_data.head()

,title,ingredients_str,instructions_str
0,Slow Cooker Chicken and Dumplings,"4 skinless, boneless chicken breast halves 2 ...","Place the chicken, butter, soup, and onion in ..."
1,Awesome Slow Cooker Pot Roast,2 (10.75 ounce) cans condensed cream of mushro...,"In a slow cooker, mix cream of mushroom soup, ..."
2,Brown Sugar Meatloaf,1/2 cup packed brown sugar 1/2 cup ketchup 1...,Preheat oven to 350 degrees F (175 degrees C)....
3,Best Chocolate Chip Cookies,"1 cup butter, softened 1 cup white sugar 1 c...",Preheat oven to 350 degrees F (175 degrees C)....
4,Homemade Mac and Cheese Casserole,8 ounces whole wheat rotini pasta 3 cups fres...,Preheat oven to 350 degrees F. Line a 2-quart ...


In [6]:
recipe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124473 entries, 0 to 124472
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   title             124473 non-null  object
 1   ingredients_str   122951 non-null  object
 2   instructions_str  124450 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


In [7]:
# Joing all the column in one string
# Define a function to combine the three columns
def combine_columns(row):
  return ' '.join([str(row['title']), str(row['ingredients_str']), str(row['instructions_str'])])

# Apply the function to each row of the dataframe
recipe_data['Combined'] = recipe_data.apply(combine_columns, axis=1)

recipe_data

,title,ingredients_str,instructions_str,Combined
0,Slow Cooker Chicken and Dumplings,"4 skinless, boneless chicken breast halves 2 ...","Place the chicken, butter, soup, and onion in ...","Slow Cooker Chicken and Dumplings 4 skinless, ..."
1,Awesome Slow Cooker Pot Roast,2 (10.75 ounce) cans condensed cream of mushro...,"In a slow cooker, mix cream of mushroom soup, ...",Awesome Slow Cooker Pot Roast 2 (10.75 ounce) ...
2,Brown Sugar Meatloaf,1/2 cup packed brown sugar 1/2 cup ketchup 1...,Preheat oven to 350 degrees F (175 degrees C)....,Brown Sugar Meatloaf 1/2 cup packed brown suga...
3,Best Chocolate Chip Cookies,"1 cup butter, softened 1 cup white sugar 1 c...",Preheat oven to 350 degrees F (175 degrees C)....,"Best Chocolate Chip Cookies 1 cup butter, soft..."
4,Homemade Mac and Cheese Casserole,8 ounces whole wheat rotini pasta 3 cups fres...,Preheat oven to 350 degrees F. Line a 2-quart ...,Homemade Mac and Cheese Casserole 8 ounces who...
...,...,...,...,...
124468,Summer Corn Salad,4 ears fresh corn 2 heads Belgian endive 2 tab...,Watch how to make this recipe.\nPreheat a gril...,Summer Corn Salad 4 ears fresh corn 2 heads Be...
124469,Zucchini Stuffed Tomatoes,4 large plum tomatoes Salt and sugar 1 1/2 pou...,Preheat the broiler. Cut the tomatoes in 1/2 c...,Zucchini Stuffed Tomatoes 4 large plum tomatoe...
124470,Pepper Pasta Quick Cook,3 tablespoons olive oil 2 tablespoons unsalted...,Heat the oil and butter in a large skillet ove...,Pepper Pasta Quick Cook 3 tablespoons olive oi...
124471,Chocolate Cake with Armagnac Ice Cream,8 ounces butter 8 ounces bittersweet chocolate...,Preheat oven to 350 degrees. On the top half o...,Chocolate Cake with Armagnac Ice Cream 8 ounce...


In [8]:
stemmer = nltk.stem.PorterStemmer()
ENGLISH_STOP_WORDS = stopwords.words('english')
additional_stop_words = [
    'make', 'dish', 'suggest', 'recipe', 'includes', 'looking', 'cook', 'something', 
    'wish', 'eat', 'help', 'find', 'mood', 'show', 'recipes', 'try', 'thinking', 
    'cooking', 'suggestions', 'recommend', 'good', 'help', 'ideas', 'inspiration',
    '.', ',', '!', '?', '-', ':', ';', '"', "'", "’", '“', '”', '(', ')'
]
STOP_words = ENGLISH_STOP_WORDS + additional_stop_words

def recipe_tokenizer(sentence):
    # remove punctuation and set to lowercase
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark, '').lower()

    # split sentence into words
    listofwords = sentence.split(' ')
    listofstemmed_words = []

    # remove stopwords and any tokens that are just empty strings
    for word in listofwords:
        if (not word in STOP_words) and (word != ''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            listofstemmed_words.append(stemmed_word)

    return listofstemmed_words

In [9]:
# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer(tokenizer=recipe_tokenizer)

# Fit and transform the data
vectorized_data = vectorizer.fit_transform(recipe_data['Combined'])

c:\Users\anura\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
vectorized_data

<124473x130224 sparse matrix of type '<class 'numpy.float64'>'
	with 10314903 stored elements in Compressed Sparse Row format>

In [11]:
pickle_out = open("vectorizer.pkl","wb")
pickle.dump(vectorizer, pickle_out)
pickle_out.close()

In [12]:
pickle_out = open("recipe_data.pkl","wb")
pickle.dump(recipe_data, pickle_out)
pickle_out.close()

In [13]:
pickle_out = open("vectorized_data.pkl","wb")
pickle.dump(vectorized_data, pickle_out)
pickle_out.close()

In [14]:
import os

In [15]:
print("Current working directory: ", os.getcwd())

Current working directory:  c:\Users\anura\OneDrive\Desktop\GitHub New\nom-nom


# Processing The User Input

In [16]:
# Process and vectorize the input
input_vector = "I wish to eat something with potato, broccoli, basil."  # Combine input words into a single string for vectorization
input_vectorized = vectorizer.transform([input_vector])

# Compute cosine similarity
cosine_sim_matrix = cosine_similarity(input_vectorized, vectorized_data)

# Get the index of the most similar recipe
most_similar_recipe_index = np.argmax(cosine_sim_matrix)

# Output the most similar recipe
print(input_vector)
#print(f"Most similar recipe: {test_df.iloc[most_similar_recipe_index]['Combined']}")
# Sort the recipes by their similarity score
top_indices = np.argsort(cosine_sim_matrix.flatten())[-1:][::-1]

#recipes_out 
from IPython.display import display, HTML

print("Here is your recipe:")
for index in top_indices:
    title = recipe_data.iloc[index]['title']
    ingredients = recipe_data.iloc[index]['ingredients_str']
    instructions = recipe_data.iloc[index]['instructions_str']
    similarity_score = f"{cosine_sim_matrix[0, index]:.2f}"
    
    # Use HTML to format the ingredients and instructions in bold
    display(HTML(f"<div><strong>Title:</strong> {title}</div>"))
    display(HTML(f"<div><strong>Ingredients:</strong> {ingredients}</div>"))
    display(HTML(f"<div><strong>Instructions:</strong> {instructions}</div>"))
    display(HTML(f"<div><strong>Similarity Score:<strong> {similarity_score}</div>"))



I wish to eat something with potato, broccoli, basil.
Here is your recipe:


In [17]:
top_indices

array([100821], dtype=int64)

In [18]:
cosine_sim_matrix

array([[0.        , 0.        , 0.        , ..., 0.03668721, 0.        ,
        0.        ]])

In [19]:
# Sort the recipes by their similarity score
top_indices = np.argsort(cosine_sim_matrix.flatten())[-10:][::-1]

# Print the top 5 most similar recipes
print("Top 5 similar recipes:")
for index in top_indices:
    print(recipe_data.iloc[index]['Combined'])

Top 5 similar recipes:
Broccoli and Cheese Smashed Potatoes 1 1/2 pounds small potatoes (recommended: Yukon gold) 1 small head broccoli 1 cup whole milk 2 tablespoons butter 1 1/2 cups shredded sharp Cheddar Salt and freshly ground black pepper Watch how to make this recipe.
Halve the potatoes and add to a large pot. Cover with water and bring to a boil over high heat. Reduce the heat to medium-low and cook the potatoes for 10 minutes while you trim and chop broccoli into 1/2-inch dice. Add the broccoli to the potatoes and cook for 5 minutes more. Drain and return to the hot pot to dry. Stir in the milk, butter and cheese and mash until the cheese melts into potatoes and broccoli. Season with salt and pepper, to taste, and transfer to a serving bowl, to serve.
Smashed Potatoes and Broccoli 4 pounds Yukon gold or russet potatoes, quartered 2 (about 6 cups) broccoli heads 1/2 cup whole milk 1/2 cup heavy cream 1/2 stick butter Salt and freshly ground black pepper Watch how to make this r

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5cb9474c-6462-4da0-a2dd-0ed9f5cafe8e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>